<h1><center>Project 2 LOS Pre-Processing</h1>
<h4>TCSS 555<br>
Spring 2018<br>
Thuan Lam, Tood Robbins, Inno Estrera</h4></center>


<h2>Libraries</h2>

In [6]:
# Load libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from dateutil.parser import parse
from datetime import datetime

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Load Data

In [7]:
import os 
cwd = os.getcwd()
print('Current folder is {}'.format(cwd))

Current folder is C:\Users\toddr\OneDrive\Desktop\Project-2-Length-Of-Stay\phase 1


In [8]:
# Load dataset
data = pd.read_csv("Data\ADMISSIONS.csv")
# data = pd.read_csv("Data\ICUSTAYS.csv")
# data = pd.read_csv("Data\PATIENTS.csv")
data = shuffle(data)
# data = data[:7000]  #get first x number of rows, going to most likely be needed for this project

# Lower case all column header
data.columns = [x.lower() for x in data.columns]

# Dimensions of dataset
print('Dataset: {} rows and {} columns'.format(data.shape[0], data.shape[1]))

Dataset: 58976 rows and 19 columns


## Preprocess Data
* <h3>histogram each column Columns

In [11]:
# shape
print('Dataset''s shape: {}'.format(data.shape))

# add compare all columns and plot the results

# head
# print(data.head(2))

# descriptions
# print(data.describe())

Datasets shape: (58976, 19)


In [12]:
print(data.head(2))

       row_id  subject_id  hadm_id            admittime            dischtime  \
26447   29516       24160   160237  2144-11-13 17:39:00  2144-11-16 13:20:00   
1603     1762        1441   169282  2158-11-14 10:30:00  2158-11-21 11:59:00   

      deathtime admission_type         admission_location discharge_location  \
26447       NaN      EMERGENCY  CLINIC REFERRAL/PREMATURE               HOME   
1603        NaN       ELECTIVE  PHYS REFERRAL/NORMAL DELI   HOME HEALTH CARE   

      insurance language           religion marital_status ethnicity  \
26447  Medicaid     ENGL  PROTESTANT QUAKER         SINGLE     WHITE   
1603    Private      NaN              OTHER        MARRIED     WHITE   

      edregtime edouttime             diagnosis  hospital_expire_flag  \
26447       NaN       NaN  FAILED BACK SYNDROME                     0   
1603        NaN       NaN    ISCHEMIC ULCER/SDA                     0   

       has_chartevents_data  
26447                     1  
1603                 

In [13]:
# checking
print(data.head(2))
print('n = {} and p = {}'.format(data.shape[0], data.shape[1]))

       row_id  subject_id  hadm_id            admittime            dischtime  \
26447   29516       24160   160237  2144-11-13 17:39:00  2144-11-16 13:20:00   
1603     1762        1441   169282  2158-11-14 10:30:00  2158-11-21 11:59:00   

      deathtime admission_type         admission_location discharge_location  \
26447       NaN      EMERGENCY  CLINIC REFERRAL/PREMATURE               HOME   
1603        NaN       ELECTIVE  PHYS REFERRAL/NORMAL DELI   HOME HEALTH CARE   

      insurance language           religion marital_status ethnicity  \
26447  Medicaid     ENGL  PROTESTANT QUAKER         SINGLE     WHITE   
1603    Private      NaN              OTHER        MARRIED     WHITE   

      edregtime edouttime             diagnosis  hospital_expire_flag  \
26447       NaN       NaN  FAILED BACK SYNDROME                     0   
1603        NaN       NaN    ISCHEMIC ULCER/SDA                     0   

       has_chartevents_data  
26447                     1  
1603                 

## Conclusion
#### From the the following tables (ADMISSIONS, ICUSTAYS, PATIENTS) this data concludes that the following columns are the most useful in determining the length of stay of the patient. 